In [1]:
from git import Repo

import pandas as pd
import os

In [2]:
path_to_data = os.path.join(os.getcwd(), 'data')
path_to_repositories = os.path.join(os.getcwd(), 'repos')

# 1. Read in repositories from CSV
repositories_metadata = pd.read_csv(os.path.join(path_to_data, 'results.csv'))

In [9]:
repositories_metadata.iloc[2]

id                                                             3277687
name                                                   feast-dev/feast
isFork                                                           False
commits                                                           3276
branches                                                            72
releases                                                           101
forks                                                              936
mainLanguage                                                    Python
defaultBranch                                                   master
license                                             Apache License 2.0
homepage                                             https://feast.dev
watchers                                                            72
stargazers                                                        5278
contributors                                                       231
size  

In [ ]:
repositories_statistics = pd.DataFrame()

for repository_metadata in repositories_metadata:
    repository_path = os.path.join(path_to_repositories, "__".join(repository_metadata["name"].split("/")))
    repo_instance = Repo.clone_from(f'https://github.com/{repository_metadata["name"]}.git', f'{repository_path}')

    os.chdir(os.path.join(path_to_data, repository_path))
    
    number_of_merge_commits = len([commit.message for commit in repo_instance.iter_commits(merges=True)])
    
    # Simply concat metadata portions that are interesting

#   a. Run analytics operations to extract relevant metrics for overall data

# 3. Print overview

# Code experimentation

In [4]:
# 2. Generate a Repo instance for it
repository_metadata = repositories_metadata.iloc[2]
repository_path = os.path.join(path_to_repositories, "__".join(repository_metadata["name"].split("/")))
repo_instance = Repo.clone_from(f'https://github.com/{repository_metadata["name"]}.git', f'{repository_path}')

os.chdir(os.path.join(path_to_data, repository_path))

#   a. Run analytics operations to extract relevant metrics for overall data

# 3. Print overview

In [5]:
os.getcwd()

'C:\\Users\\tobias.lindenbauer\\PycharmProjects\\vcs-actions-agent\\repos\\feast-dev__feast'

## Find merge commits

In [14]:
repo_instance.iter_commits().__next__().message

'docs: Operator readme fix (#4219)\n\noperator readme fix\r\n\r\nSigned-off-by: Tommy Hughes <tohughes@redhat.com>'

In [22]:
len([commit.message for commit in repo_instance.iter_commits(merges=True)])

70

## Find sequential commits modifying the same file

In [31]:
demo_repo = Repo(os.path.join(path_to_repositories, 'demo-repo'))

### Generate children (for bi-directional information in the git graph)

In [91]:
commit_children = {}
visited_commits = []

for commit in demo_repo.iter_commits(all=True):
    if commit.hexsha not in commit_children.keys():
        # Ensure we do not override already existing children that were propagated to this commit with default values
        commit_children[commit.hexsha] = {'message':commit.message, 'children_count': 0, 'children': []}
    
    for parent in commit.parents:
        # If we have already visited this commit, then its commit_children dict is initialized to default values,
        # and we can simply modify them.
        # The other case where the values are already initialized to non-default values is if a commit has  > 2 children
        # This can happen if multiple branches are create based on a commit of a commit is merged into multiple other branches.
        # In both cases there already is an entry in commit_children with this parents commit hash
        if parent.hexsha in commit_children.keys():
            # Create first child entry for this commit
            commit_children[parent.hexsha]['children_count'] += 1
            commit_children[parent.hexsha]['children'].append(commit.hexsha)
        else:
            commit_children[parent.hexsha] = {'message':commit.message, 'children_count': 1, 'children': [commit.hexsha]}
            
    visited_commits.append(commit.hexsha)

In [92]:
commit_children

{'7821ce308f797eeec65da787b96c829238e15d11': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 0,
  'children': []},
 '618b1cb31ac5069b193603e09e3147472166b663': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 3,
  'children': ['7821ce308f797eeec65da787b96c829238e15d11',
   'aa744a52fa0a7ee5b21007e64971dc2da7fa228a',
   '464e49457108e9685e0634c9da87254a85c06c07']},
 'e7f94bfac56abc6a0b408fee81c018fd220030f0': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 1,
  'children': ['7821ce308f797eeec65da787b96c829238e15d11']},
 '52fc194aff6a357c7b623fcbb9cb98561635937a': {'message': 'applied bug patch\n',
  'children_count': 1,
  'children': ['e7f94bfac56abc6a0b408fee81c018fd220030f0']},
 '533d06d3173710a8d1cb15b823cb7f9dcf72d536': {'message': 'initial bug hung\n',
  'children_count': 2,
  'children': ['52fc194aff6a357c7b623fcbb9cb98561635937a',
   '618b1cb31ac5069b193603e09e31474721

### Start with main or master

In [96]:
def clean_branch_name(branch: str) -> str:
    branch = branch.replace('*', '')
    return branch.strip()

In [97]:
branches = demo_repo.git.branch('-a').split('\n')
branches =  [clean_branch_name(branch) for branch in branches]

# Search for main or master in branches
has_main = any(('main' in branch) for branch in branches)
has_master = any(('master' in branch) for branch in branches)

In [98]:
if has_main:
    demo_repo.git.checkout('main')
elif has_master:
    demo_repo.git.checkout('master')
else:
    print('No main or master branch, skipping repo')

In [111]:
[(commit.hexsha, commit.parents) for commit in demo_repo.iter_commits(topo_sort=True)]

GitCommandError: Cmd('git') failed due to: exit code(129)
  cmdline: git rev-list --topo-sort aa744a52fa0a7ee5b21007e64971dc2da7fa228a --
  stderr: 'usage: git rev-list [<options>] <commit>... [--] [<path>...]

  limiting output:
    --max-count=<n>
    --max-age=<epoch>
    --min-age=<epoch>
    --sparse
    --no-merges
    --min-parents=<n>
    --no-min-parents
    --max-parents=<n>
    --no-max-parents
    --remove-empty
    --all
    --branches
    --tags
    --remotes
    --stdin
    --exclude-hidden=[fetch|receive|uploadpack]
    --quiet
  ordering output:
    --topo-order
    --date-order
    --reverse
  formatting output:
    --parents
    --children
    --objects | --objects-edge
    --disk-usage[=human]
    --unpacked
    --header | --pretty
    --[no-]object-names
    --abbrev=<n> | --no-abbrev
    --abbrev-commit
    --left-right
    --count
  special purpose:
    --bisect
    --bisect-vars
    --bisect-all
'

In [112]:
commit_children

{'7821ce308f797eeec65da787b96c829238e15d11': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 0,
  'children': []},
 '618b1cb31ac5069b193603e09e3147472166b663': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 3,
  'children': ['7821ce308f797eeec65da787b96c829238e15d11',
   'aa744a52fa0a7ee5b21007e64971dc2da7fa228a',
   '464e49457108e9685e0634c9da87254a85c06c07']},
 'e7f94bfac56abc6a0b408fee81c018fd220030f0': {'message': "Merge branch 'document2-bugfixes' into document2\n",
  'children_count': 1,
  'children': ['7821ce308f797eeec65da787b96c829238e15d11']},
 '52fc194aff6a357c7b623fcbb9cb98561635937a': {'message': 'applied bug patch\n',
  'children_count': 1,
  'children': ['e7f94bfac56abc6a0b408fee81c018fd220030f0']},
 '533d06d3173710a8d1cb15b823cb7f9dcf72d536': {'message': 'initial bug hung\n',
  'children_count': 2,
  'children': ['52fc194aff6a357c7b623fcbb9cb98561635937a',
   '618b1cb31ac5069b193603e09e31474721

In [107]:
sliding_window_state = []
result = []

commit_history = [commit for commit in demo_repo.iter_commits(all=True)]

def find_files_modified_within_consecutive_commits(commits_to_process, window_size, sliding_window_state):
    while commits_to_process:
        commit = commits_to_process.pop()
        
        if commit_children[commit.hexsha]['children_count'] == 1 and len(commit.parents) == 1:
            # This is an ordinary commit
            # Stay in this recursion and continue iterating
            pass
        elif commit_children[commit.hexsha]['children_count'] >= 2:
            # This commit is the starting point of at least one other branch
            result.append(find_files_modified_within_consecutive_commits(commits_to_process, window_size, sliding_window_state))
        
    

    

In [108]:
commit_history.pop().message

'init with document\n'

# Currently not used just for docs of what I tried already

In [11]:
repo_instance.git.branch()

'* (HEAD detached at origin/addFSLogging)\n  master'

In [81]:
remote_refs = repo_instance.remote().refs

In [100]:
remote = repo_instance.remote()

remote_ref = remote_refs[1]
remote.fetch(remote_ref.remote_head)
repo_instance.branches

[<git.Head "refs/heads/master">]

In [108]:
repo_instance.git.branch('--all')

'* master\n  remotes/origin/HEAD -> origin/master\n  remotes/origin/addFSLogging\n  remotes/origin/bump-chart\n  remotes/origin/bump-reqs\n  remotes/origin/codebase_structure_docs\n  remotes/origin/dependabot/pip/sdk/python/requirements/pymysql-1.1.1\n  remotes/origin/drop-uni-tests-for-py38\n  remotes/origin/feat-documentstore\n  remotes/origin/fix-odfv-test\n  remotes/origin/franciscojavierarceo-patch-1\n  remotes/origin/franciscojavierarceo-patch-2\n  remotes/origin/franciscojavierarceo-patch-3\n  remotes/origin/get-online-documents-test\n  remotes/origin/get-online-documents-tests\n  remotes/origin/go_server\n  remotes/origin/go_server_benchmarks\n  remotes/origin/hao-xu\n  remotes/origin/hao-xu-elasticsearch-vector\n  remotes/origin/hao-xu-patch\n  remotes/origin/hao-xu-patch-doc\n  remotes/origin/hao-xu-vectordb-doc\n  remotes/origin/master\n  remotes/origin/odfv-proto-change\n  remotes/origin/odfv-python-e2e-test\n  remotes/origin/pgvector-patch\n  remotes/origin/pyproject-pytho

In [72]:
for remote in repo_instance.remotes:
    remote.fetch()

In [65]:
for commit in repo_instance.iter_commits():
    print(f"{commit.hexsha} {commit.summary}")

368f671585c445052b5230efe093199edbbd5b29 Merge pull request #3387 from fgsch/patch-1
80137c293239e75380f1bdf40e90463a58e5d562 Update LICENSE
f4b63d53728515b8af24bb6dd5b11eb015f23871 chore: fix some typos in comments (#3354)
e1eabc653f80755fba24138059e0f96536414c1c Update api-key-in-query-parameter.yaml (#3384)
4c5bd64dddf7164b574997b8c09532f0eba19c37 New Published Rules - p0_security.direct-response-write-copy (#3382)
48f6e91b0b6548965f2ddb163b384d7b5a2f8770 Update webservice-ssrf.yaml (#3380)
9bc127c0cd63e679e0a208289b20706509d86330 Add rule API key in query parameter (OpenAPI Spec) (#3375)
7d2773e2118694c294a0f379d2ac83826d133649 Fix metadata for use-of-basic-authentication rule (#3378)
02489f84d65b55819c5d36aaa2a6b7b57026ea57 Merge pull request #3379 from semgrep/inkz/upd-tainted-sql-string
01ceed25f231ae196ad8e0df81c9ee3b0f059664 Merge branch 'develop' into inkz/upd-tainted-sql-string
0502383d08d39e95bc5d6983df376d2c3b825e00 New Published Rules - federicobellini.session-cookie-same